**Collaborative Filtering Approaches**
1. Memory-based:    ItemKNN
2. Model-based:     BPR, LightGCN
3. Context-based:   FM, DeepFM, WideDeep

**Content-based Approaches:** TFIDF (Cornac Models)

**Knowledge-based Approaches:** KGCN, KGAT, KGIN

**Hybrid Systems:** NeuMF

In [ ]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT
from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF


from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer, get_model

# configurations initialization
config = Config(model='BPR', dataset='ml-100k')

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# write config info into log
logger.info(config)

# dataset creating and filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# model loading and initialization
model = BPR(config, train_data._dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
trainer = trainer_class(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

# model evaluation
test_result = trainer.evaluate(test_data)
print(test_result)

In [ ]:
from recbole.quick_start import run_recbole

run_recbole(model='LightGCN', dataset='ml-100k')

In [ ]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation


from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

models = {
    'BPR': BPR,
    # 'LightGCN': LightGCN,
    # 'ItemKNN': ItemKNN, 
    # 'FM': FM,
    # 'DeepFM': DeepFM,
    # 'WideDeep': WideDeep,
    # 'KGCN': KGCN,
    # 'KGIN': KGIN,
    # 'KGAT': KGAT,
    # 'NeuMF': NeuMF
    }

for model_name, model_class in models.items():
    print(f"\033[93mRunning model: {model_name}\033[0m")

    

    config = Config(model=model_name, dataset='ml-100k')

    hp = HyperTuning(objective_function=objective_function, algo='exhaustive', early_stop=10,
                max_evals=100, params_file='params.hyper', fixed_config_file_list=config)

    # run
    hp.run()
    # export result to the file
    hp.export_result(output_file='hyper_example.result')
    # print best parameters
    print('best params: ', hp.best_params)
    # print best result
    print('best result: ')
    print(hp.params2result[hp.params2str(hp.best_params)])


    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = model_class(config, train_data._dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
    trainer = trainer_class(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    print(test_result)

In [ ]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

models = {
    'BPR': BPR,
    # other models commented out
}

for model_name, model_class in models.items():
    print(f"\033[93mRunning model: {model_name}\033[0m")

    config = Config(model=model_name, dataset='ml-100k')
    config_dict = {'model': model_name, 'dataset': 'ml-100k'}
    # Initialize hyperparameter tuning for the model
    hp = HyperTuning(
        objective_function=objective_function,
        algo='exhaustive',
        early_stop=10,
        max_evals=100,
        params_file='params.hyper',
        fixed_config_file_list=['config.yaml']  # Pass config object here
        # fixed_config_file_list=[config]  # Pass config object here
    )

    # Run the hyperparameter tuning
    hp.run()
    hp.export_result(output_file='hyper_example.result')

    print('best params: ', hp.best_params)
    print('best result: ')
    print(hp.params2result[hp.params2str(hp.best_params)])

    # Now run training with best hyperparameters
    

    # Seed and logger
    init_seed(config['seed'], config['reproducibility'])
    init_logger(config)
    logger = getLogger()
    logger.info(config)

    # Dataset loading and preparation
    dataset = create_dataset(config)
    logger.info(dataset)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Model initialization
    model = model_class(config, train_data._dataset).to(config['device'])
    logger.info(model)

    # Trainer initialization and training
    trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
    trainer = trainer_class(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

    # Evaluation
    test_result = trainer.evaluate(test_data)
    print(test_result)


In [ ]:
# @Time   : 2023/2/13
# @Author : Gaowei Zhang
# @Email  : zgw2022101006@ruc.edu.cn


import argparse

from recbole.quick_start import run
from recbole.utils import list_to_latex

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_list", "-m", type=str, default="BPR", help="name of models"
    )
    parser.add_argument(
        "--dataset", "-d", type=str, default="ml-100k", help="name of datasets"
    )
    parser.add_argument("--config_files", type=str, default=None, help="config files")
    parser.add_argument(
        "--valid_latex", type=str, default="./latex/valid.tex", help="config files"
    )
    parser.add_argument(
        "--test_latex", type=str, default="./latex/test.tex", help="config files"
    )
    parser.add_argument(
        "--nproc", type=int, default=1, help="the number of process in this group"
    )
    parser.add_argument(
        "--ip", type=str, default="localhost", help="the ip of master node"
    )
    parser.add_argument(
        "--port", type=str, default="5678", help="the port of master node"
    )
    parser.add_argument(
        "--world_size", type=int, default=-1, help="total number of jobs"
    )
    parser.add_argument(
        "--group_offset",
        type=int,
        default=0,
        help="the global rank offset of this group",
    )

    args, _ = parser.parse_known_args()

    model_list = args.model_list.strip().split(",")
    config_file_list = (
        args.config_files.strip().split(" ") if args.config_files else None
    )
    valid_file = args.valid_latex.strip()
    test_file = args.test_latex.strip()

    valid_result_list = []
    test_result_list = []

    run_times = len(model_list)

    for idx in range(run_times):
        model = model_list[idx]

        valid_res_dict = {"Model": model}
        test_res_dict = {"Model": model}
        result = run(
            model,
            args.dataset,
            config_file_list=config_file_list,
            nproc=args.nproc,
            world_size=args.world_size,
            ip=args.ip,
            port=args.port,
            group_offset=args.group_offset,
        )
        valid_res_dict.update(result["best_valid_result"])
        test_res_dict.update(result["test_result"])
        bigger_flag = result["valid_score_bigger"]
        subset_columns = list(result["best_valid_result"].keys())

        valid_result_list.append(valid_res_dict)
        test_result_list.append(test_res_dict)

    df_valid, tex_valid = list_to_latex(
        convert_list=valid_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )
    df_test, tex_test = list_to_latex(
        convert_list=test_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )

    with open(valid_file, "w") as f:
        f.write(tex_valid)
    with open(test_file, "w") as f:
        f.write(tex_test)

In [ ]:
def main():
    # Set values directly
    model_list = "BPR".strip().split(",")
    dataset = "ml-100k"
    config_file_list = None  # or ["path/to/config1.yaml", "path/to/config2.yaml"]
    valid_file = "./latex/valid.tex"
    test_file = "./latex/test.tex"
    nproc = 1
    ip = "localhost"
    port = "5678"
    world_size = -1
    group_offset = 0

    valid_result_list = []
    test_result_list = []

    run_times = len(model_list)

    for idx in range(run_times):
        model = model_list[idx]

        valid_res_dict = {"Model": model}
        test_res_dict = {"Model": model}

        result = run(
            model,
            dataset,
            config_file_list=config_file_list,
            nproc=nproc,
            world_size=world_size,
            ip=ip,
            port=port,
            group_offset=group_offset,
        )

        valid_res_dict.update(result["best_valid_result"])
        test_res_dict.update(result["test_result"])
        bigger_flag = result["valid_score_bigger"]
        subset_columns = list(result["best_valid_result"].keys())

        valid_result_list.append(valid_res_dict)
        test_result_list.append(test_res_dict)

    df_valid, tex_valid = list_to_latex(
        convert_list=valid_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )
    df_test, tex_test = list_to_latex(
        convert_list=test_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )

    # with open(valid_file, "w") as f:
    #     f.write(tex_valid)
    # with open(test_file, "w") as f:
    #     f.write(tex_test)


In [ ]:
main()

In [ ]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.model.general_recommender import BPR, LightGCN, ItemKNN, NeuMF
from recbole.model.context_aware_recommender import FM, DeepFM, WideDeep
from recbole.model.knowledge_aware_recommender import KGCN, KGIN, KGAT

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger, get_trainer

def main():
    # === Configuration ===
    model_list = ["BPR"]
    dataset_name = "ml-100k"
    fixed_config_files = [r"C:\Users\Sharjeel Mustafa\Documents\Documents\Academic\C874\C874-final-project\config\config.yaml"]
    valid_file = "./latex/valid.tex"
    test_file = "./latex/test.tex"

    valid_result_list = []
    test_result_list = []

    for model_name in model_list:
        print(f"\033[93mRunning model: {model_name}\033[0m")

        # Step 1: Hyperparameter Tuning
        hp = HyperTuning(
            objective_function=objective_function,
            algo='exhaustive',
            early_stop=10,
            max_evals=100,
            params_file='C:/Users/Sharjeel Mustafa/Documents/Documents/Academic/C874/C874-final-project/config/model.hyper',
            fixed_config_file_list=fixed_config_files
        )

        hp.run()
        hp.export_result(output_file='hyper_example.result')

        print("Best Params: ", hp.best_params)
        print("Best Result: ")
        print(hp.params2result[hp.params2str(hp.best_params)])

        # Step 2: Re-train using best parameters
        config = Config(model=model_name, dataset=dataset_name, config_dict=hp.best_params)
        init_seed(config["seed"], config["reproducibility"])
        init_logger(config)
        logger = getLogger()
        logger.info(config)

        dataset = create_dataset(config)
        logger.info(dataset)

        train_data, valid_data, test_data = data_preparation(config, dataset)

        model_class = config['model']
        model = config.model_class(config, train_data._dataset).to(config['device'])
        logger.info(model)

        trainer_class = get_trainer(config["MODEL_TYPE"], config["model"])
        trainer = trainer_class(config, model)

        best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
        test_result = trainer.evaluate(test_data)

        # For LaTeX summary
        valid_result_list.append({"Model": model_name, **best_valid_result})
        test_result_list.append({"Model": model_name, **test_result})
        bigger_flag = config["valid_metric_bigger"]
        subset_columns = list(best_valid_result.keys())

        print("\nTest Result:")
        print(test_result)

    # Step 3: Generate and write LaTeX outputs
    df_valid, tex_valid = list_to_latex(
        convert_list=valid_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )
    df_test, tex_test = list_to_latex(
        convert_list=test_result_list,
        bigger_flag=bigger_flag,
        subset_columns=subset_columns,
    )

    with open(valid_file, "w") as f:
        f.write(tex_valid)
    with open(test_file, "w") as f:
        f.write(tex_test)
